In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import gradient_descent_v2
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

def eachFile(filepath):
	pathDir = os.listdir(filepath)
	out = []
	for allDir in pathDir:
		child = allDir
		out.append(child)
	return out


NUM_CLASSES = 11
TRAIN_PATH = 'dataset/train/'
TEST_PATH = 'dataset/test/'

FC_NUMS = 4096

FREEZE_LAYERS = 17

IMAGE_SIZE = 224

base_model = VGG19(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(4096, activation='relu')(x)
prediction = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=prediction)

for layer in model.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in model.layers[FREEZE_LAYERS:]:
    layer.trainable = True

model.compile(optimizer=gradient_descent_v2.SGD(learning_rate=0.001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
train_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(directory=TRAIN_PATH,
                                                    target_size=(IMAGE_SIZE, IMAGE_SIZE), classes=eachFile(TRAIN_PATH))
filepath = 'model/sewer_weight.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

history_ft = model.fit_generator(train_generator, epochs=10, callbacks=[checkpoint])
model.save('model/sewer_weight_final.h5')

In [7]:
from dataclasses import dataclass
import os

class NUM_class:
    class_A: int = 0;
    class_B: int = 0;
    class_C: int = 0;

class Defect_info:
    Img_loc: str;
    defect: str;
    Class: chr;

class pipe_info:
    type: str;
    length: float;
    diameter: float;

pipe_cost = 0;

NUM_total_img = 1;

NUM_complete_pipe = 0;

def count_gauge(NUM_total_img, NUM_complete_pipe):
    calc_progress = NUM_complete_pipe % NUM_total_img;
    
    return calc_progress;

#calc_progress = count_gauge(NUM_total_img, NUM_complete_pipe);

def calculate_risk(NUM_class):
    Defect_sum = 0;
    
    if NUM_class.class_A != 0:
        return 1;
    Defect_sum += NUM_class.class_B * 20;
    Defect_sum += NUM_class.class_C * 5;
    
    if Defect_sum >= 100:
        return 1;
    else:
        return 0;

#threshold = calculate_risk(NUM_class);

def calculate_cost(pipe_info):
    pipe_cost = 0.0;
    
    if pipe_info.type == "CONC":
        pipe_cost=  pipe_info.length * pipe_info.diameter * 512740;
    elif pipe_info.type == "PLA":
        pipe_cost=  pipe_info.length * pipe_info.diameter * 707247;
    #더 추가
    return pipe_cost;

#pipe_cost = calculate_cost(pipe_info);

In [ ]:

from tensorflow.python.keras.initializers import glorot_uniform
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import cv2
from keras.utils.generic_utils import CustomObjectScope
from keras.preprocessing.image import  img_to_array
from dataclasses import dataclass
#import sewer_defect


os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

# image folder
folder_path = 'dataset/test/'

# path to model
model_path = 'model/sewer_weight.h5'
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model(model_path)

# dimensions of images
img_width, img_height = 224, 224

i = 0
images = []
for img in os.listdir(folder_path):
   img1 = image.load_img(os.path.join(folder_path, img), target_size=(img_width, img_height))
   img2 = img_to_array(img1)
   img2 = np.expand_dims(img2, axis=0)
   classes = model.predict(img2)[0]
   idxs = np.argsort(classes)[::-1][:2]

   classname = ['BK', 'CC', 'CL', 'DS', 'ETC', 'JD', 'JF', 'LP',
                'SD', 'UP_IN', 'UP_PJ']

   out = cv2.imread(os.path.join(folder_path, img))

   for (i, j) in enumerate(idxs):
        if classes[idxs[i]] * 100 <= 5:
           continue;
        label = "{}:{:.2f}%".format(classname[idxs[i]], classes[idxs[i]] * 100)
        cv2.putText(out, label, (10, (i * 30) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        #numclass로 class 분류하여 저장
    
   cv2.imwrite("visualization/sd/%s"%img,out)
   

In [4]:
from tensorflow.python.keras.initializers import glorot_uniform
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import cv2
from keras.utils.generic_utils import CustomObjectScope
from keras.preprocessing.image import  img_to_array

def detect_defect(img, model):
    os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

    # image folder
    folder_path = 'dataset/test/'

    # path to model
    #model_path = 'model/sewer_weight.h5'
    #with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
    #        model = load_model(model_path)

    # dimensions of images
    img_width, img_height = 224, 224
    i = 0
    images = []
    img1 = image.load_img(os.path.join(folder_path, img), target_size=(img_width, img_height))
    img2 = img_to_array(img1)
    img2 = np.expand_dims(img2, axis=0)
    classes = model.predict(img2)[0]
    idxs = np.argsort(classes)[::-1][:2]

    classname = ['BK', 'CC', 'CL', 'DS', 'ETC', 'JD', 'JF', 'LP',
                'SD', 'UP_IN', 'UP_PJ']

    out = cv2.imread(os.path.join(folder_path, img))

    for (i, j) in enumerate(idxs):
        if classes[idxs[i]] * 100 <= 5:
            continue;
        label = "{}:{:.2f}%".format(classname[idxs[i]], classes[idxs[i]] * 100)
        
        cv2.putText(out, label, (10, (i * 30) + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    cv2.imwrite("visualization/sd/%s"%img,out)
   

In [5]:
from tensorflow.python.keras.initializers import glorot_uniform
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import cv2
from keras.utils.generic_utils import CustomObjectScope
from keras.preprocessing.image import  img_to_array
from dataclasses import dataclass

os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3"

# image folder
folder_path = 'dataset/test/'

# path to model
model_path = 'model/sewer_weight.h5'
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        model = load_model(model_path)
        
for img in os.listdir(folder_path):
    detect_defect(img, model);
    

1


2022-05-18 01:42:51.664304: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


UP_PJ:100.00%
1
UP_PJ:100.00%
1
SD:93.86%
CL:5.10%
1
CL:90.02%
JF:8.28%
1
UP_PJ:100.00%
1
UP_PJ:100.00%
1
JF:59.33%
CL:40.11%
1
CL:74.06%
JD:25.17%
1
BK:99.27%
1
LP:100.00%
1
LP:100.00%
1
BK:99.36%
1
BK:90.08%
1
LP:100.00%
1
LP:100.00%
1
JF:69.91%
LP:27.13%
1
BK:100.00%
1
ETC:99.97%
1
ETC:100.00%
1
SD:99.53%
1
ETC:99.85%
1
CL:67.74%
SD:32.23%
1
ETC:100.00%
1
SD:99.80%
1
ETC:99.99%
1
ETC:100.00%
1
JF:100.00%
1
JF:100.00%
1
JF:55.93%
CL:42.12%
1
CC:100.00%
1
JF:100.00%
1
JF:100.00%
1
CC:100.00%
1
SD:77.68%
UP_IN:17.24%
1
UP_IN:99.95%
1
SD:99.59%
1
SD:99.60%
1
CC:100.00%
1
JF:100.00%
1
JF:100.00%
1
UP_IN:93.25%
UP_PJ:6.05%
1
CC:100.00%
1
SD:97.20%
1
JD:100.00%
1
DS:100.00%
1
JD:100.00%
1
UP_IN:100.00%
1
BK:99.89%
1
LP:100.00%
1
LP:100.00%
1
BK:99.91%
1
UP_IN:100.00%
1
JD:99.90%
1
DS:100.00%
1
DS:100.00%
1
UP_IN:100.00%
1
CL:100.00%
1
UP_IN:100.00%
1
DS:100.00%
1
JD:99.55%
1
DS:100.00%
1
BK:100.00%
1
CL:99.76%
1
CL:97.40%
1
CL:100.00%
1
LP:92.42%
CL:5.17%
1
DS:100.00%
1
JD:99.98%
1
JF:84.1